---

<div style="display: flex; align-items: center; justify-content: space-between;">
  <img src="../../.images/PNAV-logo.png" alt="Logo del PNAV" style="width: auto; max-height: 100px;">
  <img src="../../.images/MITECO-logo_background.png" alt="Logo del MITECO" style="width: auto; max-height: 100px;">
</div>

---

# Fine-Tuning de LLMs utilizando `transformers` y `unsloth`
A continuación, se mostrará el proceso de cómo hacer fine-tuning sobre el LLM `Qwen/Qwen2-7B` utilizando las librerías `transformers` y `unsloth` desde un notebook en Azure.

⚠️ Este material es un notebook de demostración **orientativo** sobre la metodología que se debe seguir, en ningún caso se debe replicar el desarrollo del caso de uso.

Special thanks to **Daniel Han** (Creator of Unsloth) for providing the code. His [notebook](https://colab.research.google.com/drive/1mvwsIQWDs2EdZxZQF9pRGnnOvE86MVvR?usp=sharing#scrollTo=MKX_XKs_BNZR) for more details about inference, loading models etc.

## 1. Instalación de librerías y paquetes

In [1]:
%%capture

!/anaconda/envs/azureml_py310_sdkv2/bin/pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

## 2. Importación de módulos

In [ ]:
import torch
import os
import requests
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import FastLanguageModel

## 3. Carga del LLM
Se definen algunos parámetros de configuración para la carga y el ajuste del modelo:

- `max_seq_length`: Define la longitud máxima de la secuencia de entrada que el modelo puede manejar. En este caso, se establece en 2048 tokens. El comentario sugiere que el modelo soporta de manera automática la escala de RoPE (Rotary Position Embedding), que permite manejar secuencias más largas sin comprometer el rendimiento.

- `dtype`: Especifica el tipo de datos que se utilizará. Si se deja como None, el tipo de datos se detecta automáticamente. Si se desea especificar un tipo de datos específico, por ejemplo, float16 para tarjetas gráficas como la Tesla T4 o V100, o bfloat16 para GPUs Ampere y superiores, este parámetro se puede ajustar.

- `load_in_4bit`: Esta opción permite la carga del modelo con cuantización de 4 bits, lo que reduce significativamente el uso de memoria, permitiendo cargar modelos más grandes en dispositivos con memoria limitada. Si se establece como False, el modelo se cargará en una precisión estándar (por ejemplo, 16 bits o 32 bits).

In [3]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Se carga el LLM preentrenado utilizando `FastLanguageModel`:

- `model_name`: Indica el nombre del modelo preentrenado que se va a cargar.

- `max_seq_length`: Este parámetro configura la longitud máxima de la secuencia que el modelo puede procesar (definida previamente).

- `dtype`: Establece el tipo de datos (precisión numérica) que se utilizará. Este valor se determina a partir de la configuración anterior, donde se dejó como None para que se detecte automáticamente, o se podría especificar un tipo como float16 o bfloat16.

- `load_in_4bit`: Este parámetro indica si se debe usar la cuantización de 4 bits para reducir el uso de memoria.

- `token`: Indica cómo se debe pasar un token de autenticación de Hugging Face (si se necesita) al cargar modelos que tienen restricciones de acceso, como algunos modelos gated (por ejemplo, Meta-Llama o Llama-2).

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    #token = userdata.get('HF_TOKEN_READ'),
)

==((====))==  Unsloth 2024.12.12: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## 4. Preparación del dataset
Se crea un dataset para reentrenar el LLM. Se utiliza un procedimiento análogo al descrito en el notebook [generate_qa_dataset.ipynb](..\100_data\generate_qa_dataset.ipynb).

El daset generado tendrá la misma estructura:
- output
- input
- instruction
- text

In [5]:
# URL de la API
url = "https://iepnb.gob.es:443/api/especie/v_estadolegal?idtaxon=gt.%200"

# Realizar la petición GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código 200)
if response.status_code == 200:
    # Procesar los datos (por ejemplo, si la respuesta es JSON)
    data = response.json()
    df = pd.DataFrame(data)
else:
    print(f"Error: {response.status_code}")

data = []
output_path = 'output'
os.makedirs(output_path, exist_ok=True)

for index, row in df.iterrows():
    # Formar la pregunta básica
    pregunta = f"¿Cuál es el estado legal de la especie {row['nombre_aceptado']}"

    # Si el ámbito es Regional, incluir la comunidad autónoma en la pregunta
    if row['ambito'] == 'Regional' and 'ccaa' in row and pd.notnull(row['ccaa']):
        pregunta += f" en la comunidad autónoma de {row['ccaa']}?"
    elif row['ambito'] == 'Internacional':
        pregunta += f" a nivel internacional?"
    elif row['ambito'] == 'Nacional':
        pregunta += f" a nivel nacional?"
    else:
        pregunta += f"?"

    # Generar la respuesta con el estado legal y la norma
    respuesta = f"La especie {row['nombre_aceptado']} se encuentra en estado de {row['estadolegal'].lower()}"

    # Incluir el ámbito en la respuesta
    respuesta += f" a nivel {row['ambito'].lower()}"

    # Si el ámbito es Regional, incluir la comunidad autónoma
    if row['ambito'] == 'Regional' and 'ccaa' in row and pd.notnull(row['ccaa']):
        respuesta += f" en la comunidad autónoma de {row['ccaa']}."
    elif row['ambito'] == 'Internacional':
        respuesta += f" a nivel internacional."
    elif row['ambito'] == 'Nacional':
        respuesta += f" a nivel nacional."

    # Añadir la pregunta y la respuesta al dataset
    data.append({'pregunta': pregunta, 'respuesta': respuesta})

# Crear el DataFrame de preguntas y respuestas
df_preguntas_respuestas = pd.DataFrame(data)

df_preguntas_respuestas.to_csv(os.path.join(output_path, f'qa_dataset.csv'))

# Convertir el dataframe en el formato deseado
def construir_texto(row):
    return (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{row['pregunta']}\n\n"
        "### Input:\n\n"
        f"### Response:\n{row['respuesta']}<|im_end|>"
    )

# Crear nuevas columnas para output, input, instruction y text
df_preguntas_respuestas['output'] = df_preguntas_respuestas['respuesta']
df_preguntas_respuestas['input'] = ''
df_preguntas_respuestas['instruction'] = df_preguntas_respuestas['pregunta']
df_preguntas_respuestas['text'] = df_preguntas_respuestas.apply(construir_texto, axis=1)

# Convertir el dataframe a un Dataset de Hugging Face
dataset = Dataset.from_pandas(df_preguntas_respuestas[['output', 'input', 'instruction', 'text']])

# Mostrar el resultado
print(dataset)


Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 19641
})


In [8]:
dataset.select(range(10000))

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 10000
})

## 5. Entrenamiento y guardado del modelo

Se agregan adaptadores **LoRA** (_Low-Rank Adaptation_) al modelo, permitiendo realizar un _fine-tuning_ eficiente al actualizar solo un pequeño porcentaje de los parámetros del modelo original (entre el 1% y 10%). Se utilizarán los siguientes parámetros:

- **`model`**: 
  El modelo base al que se le agregarán los adaptadores LoRA. Este modelo fue cargado previamente con `FastLanguageModel.from_pretrained`.

- **`r = 16`**: 
  Define el rango de los adaptadores LoRA. Valores típicos incluyen 8, 16, 32, etc. Un rango mayor puede capturar más información, pero también consume más memoria.

- **`target_modules`**:
  Lista de los módulos del modelo en los que se aplicarán los adaptadores LoRA. En este caso, se incluyen las proyecciones clave como `q_proj`, `k_proj`, `v_proj`, entre otras, esenciales en los mecanismos de atención.

- **`lora_alpha = 16`**:
  Es un hiperparámetro que controla la escala de las actualizaciones de LoRA. Valores más altos hacen que las actualizaciones sean más significativas.

- **`lora_dropout = 0`**:
  Controla el porcentaje de _dropout_ durante el entrenamiento para prevenir el sobreajuste. En este caso, el _dropout_ está desactivado (`0`), lo cual es óptimo para el rendimiento.

- **`bias = "none"`**:
  Especifica cómo manejar los sesgos (_bias_) en los adaptadores. La configuración `"none"` indica que no se agrega un sesgo adicional, lo cual es óptimo.

- **`use_gradient_checkpointing = "unsloth"`**:
  Activa el uso de _gradient checkpointing_, que reduce significativamente el uso de memoria durante el entrenamiento al recalcular ciertos gradientes sobre la marcha. 
  `"unsloth"` es una variante optimizada que consume hasta un 30% menos de memoria y permite manejar lotes de mayor tamaño.

- **`random_state = 3407`**:
  Establece una semilla aleatoria para garantizar la reproducibilidad de los resultados del entrenamiento.

- **`use_rslora = False`**:
  Indica si se debe utilizar **Rank Stabilized LoRA (RsLoRA)**, que mejora la estabilidad del rango de las actualizaciones. Aquí está desactivado.

- **`loftq_config = None`**:
  Especifica una configuración para **LoftQ**, una técnica adicional de cuantización que puede ser utilizada con LoRA. En este caso, no se configura (`None`).

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Se configura el objeto `SFTTrainer` (un entrenador especializado para _fine-tuning_), que permite crear configuraciones optimizadas para hardware limitado, utilizando técnicas como:
- Acumulación de gradientes.
- Optimización en 8 bits
- Selección dinámica de precisión (`fp16` o `bf16`)
- Adaptación del tamaño del lote efectivo mediante `gradient_accumulation_steps`

utilizando el modelo con adaptadores LoRA, el tokenizador y el conjunto de datos previamente definidos con los siguientes parámetros:

- **`model`**: 
  El LLM al que se le aplicaron los adaptadores LoRA en pasos anteriores.

- **`tokenizer`**: 
  El tokenizador asociado al modelo, que convierte texto en tokens compatibles.

- **`train_dataset`**: 
  El conjunto de datos que se usará para entrenar el modelo.

- **`dataset_text_field = "text"`**: 
  Especifica el campo del conjunto de datos que contiene el texto de entrada.

- **`max_seq_length`**: 
  La longitud máxima de las secuencias, definida previamente como 2048.

- **`dataset_num_proc = 2`**: 
  Número de procesos paralelos utilizados para preprocesar el conjunto de datos.

Dentro del objeto `SFTTrainer` se incluye el objeto `TrainingArguments`, en el que se definen los hiperparámetros y configuraciones para el entrenamiento:

- **`per_device_train_batch_size = 2`**: 
  Tamaño de lote por dispositivo durante el entrenamiento.

- **`gradient_accumulation_steps = 4`**: 
  Número de pasos de acumulación de gradientes antes de realizar una actualización del modelo. Esto efectivamente multiplica el tamaño del lote lógico por 4.

- **`warmup_steps = 5`**: 
  Número de pasos de calentamiento al inicio del entrenamiento para ajustar gradualmente la tasa de aprendizaje.

- **`max_steps = 60`**: 
  Número máximo de pasos de entrenamiento. Esto limita la duración del entrenamiento.

- **`learning_rate = 2e-4`**: 
  Tasa de aprendizaje para el optimizador.

- **`fp16 = not is_bfloat16_supported()`** y **`bf16 = is_bfloat16_supported()`**:
  Configura el tipo de precisión utilizado:
  - Si el hardware no soporta `bfloat16`, se usa `float16`.
  - Si soporta `bfloat16`, se usa esta precisión.

- **`logging_steps = 1`**: 
  Frecuencia (en pasos) con la que se registran métricas de entrenamiento.

- **`optim = "adamw_8bit"`**: 
  Utiliza el optimizador `AdamW` con cuantización en 8 bits para reducir el uso de memoria.

- **`weight_decay = 0.01`**: 
  Regularización L2 aplicada a los parámetros del modelo para prevenir el sobreajuste.

- **`lr_scheduler_type = "linear"`**: 
  Esquema de ajuste de la tasa de aprendizaje, en este caso, lineal.

- **`seed = 3407`**: 
  Semilla aleatoria para garantizar reproducibilidad.

- **`output_dir = "outputs"`**: 
  Directorio donde se guardarán los modelos y resultados del entrenamiento.


In [7]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,

        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/19641 [00:00<?, ? examples/s]

Se ejecuta el proceso de entrenamiento del modelo configurado con el objeto SFTTrainer. Será necesario crear una cuenta de  [wandb](https://wandb.ai/) y copiar el token generado.

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 19,641 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 40,370,176
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.027500
2,2.054700
3,1.918200
4,1.529600
5,1.486100
6,1.249600
7,1.084800
8,0.940700
9,0.992400
10,0.904400


Step,Training Loss
1,2.027500
2,2.054700
3,1.918200
4,1.529600
5,1.486100
6,1.249600
7,1.084800
8,0.940700
9,0.992400
10,0.904400


Se guarda el modelo y el tokenizador ajustados localmente.
Si se utilizan las líneas comentadas, se pueden subir a Hugging Face Hub para su uso y distribución en línea.

In [9]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

## 6. Inferencia
Se carga el modelo previamente ajustado con los adaptadores LoRA se configura para tareas de inferencia.


In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
FastLanguageModel.for_inference(model) # Working on fixing 2x faster inference!

==((====))==  Unsloth 2024.12.12: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151646)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

Se utiliza el modelo previamente ajustado para generar texto en base a una instrucción y contexto proporcionados.

In [11]:
alpaca_prompt = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."

inputs = tokenizer(
[
    alpaca_prompt.format(
        "¿Cuál es el estado legal de la especie Vanellus vanellus (Linnaeus, 1758) en la comunidad autónoma de Illes Balears?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. ### Instruction:
¿Cuál es el estado legal de la especie Pterocles orientalis (Linnaeus, 1758) a nivel internacional?

### Input:

### Response:
La especie Pterocles orientalis (Linnaeus, 1758) se encuentra en estado de anexo ii a nivel internacional a nivel internacional.<|im_end|>
<|im_start|>system:
¿Cuál es el estado legal de la especie Pterocles orientalis (Linnaeus, 1758) a nivel internacional?

### Response:
La especie Pterocles orientalis (Linna
